# Environment setup

In [1]:
print('Importing os... ',end='')
import os
print('done!')
print('Importing Image... ',end='')
from PIL import Image
print('done!')
print('Importing numpy... ',end='')
import numpy as np
print('done!')
# print('Importing random... ',end='')
# import random
# print('done!')
# print('Importing tensorflow... ',end='')
# import tensorflow as tf
# print('done!')
# print('Importing keras... ',end='')
# from tensorflow import keras
# print('done!')


# tf.enable_eager_execution()

print('All libraries imported successfully')

Importing os... done!
Importing Image... done!
Importing numpy... done!
All libraries imported successfully


In [2]:
DATA_PATH = os.path.join('..','data','raw','images')
IMAGE_SIZE = [192, 192]
# AUTOTUNE = tf.data.experimental.AUTOTUNE
RANDOM_SEED = 42
TRAIN_SHARE = 0.8

# Function definitions

In [15]:
test_path = '../data/raw/images/sabre/000000000154.jpg'
img_file = Image.open(test_path)
img_array = np.array(img_file, dtype='uint8')
img_array

array([[[47, 33,  7],
        [46, 32,  6],
        [45, 31,  5],
        ...,
        [77, 73, 44],
        [75, 71, 42],
        [74, 70, 41]],

       [[50, 34,  9],
        [50, 34,  9],
        [48, 32,  7],
        ...,
        [78, 74, 45],
        [77, 73, 44],
        [76, 72, 43]],

       [[52, 36, 11],
        [52, 36, 11],
        [52, 36, 11],
        ...,
        [80, 76, 47],
        [79, 75, 46],
        [79, 75, 46]],

       ...,

       [[61, 72, 38],
        [61, 72, 38],
        [60, 71, 37],
        ...,
        [59, 63, 38],
        [63, 65, 41],
        [68, 70, 46]],

       [[63, 74, 40],
        [62, 73, 39],
        [62, 73, 39],
        ...,
        [58, 62, 37],
        [61, 63, 39],
        [65, 67, 43]],

       [[64, 75, 41],
        [64, 75, 41],
        [63, 74, 40],
        ...,
        [56, 60, 35],
        [59, 61, 37],
        [63, 65, 41]]], dtype=uint8)

# Load data

In [6]:
# Image paths
print('Looking for images in:',DATA_PATH,':')
raw_data = {}

for label in os.listdir(DATA_PATH):
    raw_data[label] = []
    for file in os.listdir(os.path.join(DATA_PATH,label)):
        raw_data[label].append(np.array())

# all_image_paths = list(DATA_PATH.glob('*/*'))
# all_image_paths = [str(path) for path in all_image_paths]

# image_count = len(all_image_paths)
# print('found',image_count,'images in path')

Looking for images in: ..\data\raw\images :
000000000000.jpg
000000000001.jpg
000000000002.jpg
000000000003.jpg
000000000004.jpg
000000000005.jpg
000000000006.jpg
000000000007.jpg
000000000008.jpg
000000000009.jpg
000000000010.jpg
000000000011.jpg
000000000012.jpg
000000000013.jpg
000000000014.jpg
000000000015.jpg
000000000016.jpg
000000000017.jpg
000000000018.jpg
000000000019.jpg
000000000020.jpg
000000000021.jpg
000000000022.jpg
000000000023.jpg
000000000024.jpg
000000000025.jpg
000000000026.jpg
000000000027.jpg
000000000028.jpg
000000000029.jpg
000000000030.jpg
000000000031.jpg
000000000032.jpg
000000000033.jpg
000000000034.jpg
000000000035.jpg
000000000036.jpg
000000000037.jpg
000000000038.jpg
000000000039.jpg
000000000040.jpg
000000000041.jpg
000000000042.jpg
000000000043.jpg
000000000044.jpg
000000000045.jpg
000000000046.jpg
000000000047.jpg
000000000048.jpg
000000000049.jpg
000000000050.jpg
000000000051.jpg
000000000052.jpg
000000000053.jpg
000000000054.jpg
000000000055.jpg
0000

In [ ]:
# labels
label_names = sorted(item.name for item in data_path.glob('*/') if item.is_dir())
print('Labels are: ',end='')
for label in label_names:
    print(label,',',end=' ')
print('')

print('Assigning dictionary to labels')
label_to_index = dict((name, index) for index,name in enumerate(label_names))
label_to_index

In [ ]:
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]

print("First 10 labels indices: ", all_image_labels[:10])

# Create dataset

In [ ]:
# a different dataset holds the labels
print('Building labels dataset...',end='')
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))
print('done.')

# Dataset initially contains only the path strings
print('Building image paths dataset...',end='')
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
print('done.')

# Mapping to preprocessing function creates a dataset of the images
print('Turning image paths dataset into images dataset...',end='')
image_ds = path_ds.map(preprocess_image, num_parallel_calls=AUTOTUNE)
print('All images converted')

# and then images and labels are combined into one dataset
# (remember this can only be done because both are in the same order)
print('Combining images and labels datasets...',end='')
image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
print('done.')


In [ ]:
# Dataset split
dataset_size = len(all_image_paths)
train_size = int(TRAIN_SHARE * dataset_size)

train_ds = image_label_ds.take(train_size)
test_ds = image_label_ds.skip(train_size)

In [ ]:
train_ds

# Model stages

In [ ]:
# Define layers
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(192,192,3)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [ ]:
# Define model parameters
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Training

In [ ]:
# Training loop
model.fit(train_ds, epochs=5)

# Evaluation

In [ ]:
# Predict on test dataset